<div style="background-color:black;color:white;padding:20px">

<center><h3> LAA </h3></center>
<center><h1> Projeto Bicóptero </h1></center>

<center> <h1> Modelo dinâmico do conjunto braço robótico-bicóptero usando a formulação de Lagrange</h1></center>

<h3> Por Gabriel Henrique Costa e Silva </h3> 
</div>

<h2> Inicializando as bibliotecas </h2>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import * 
from sympy import trigsimp, pi, cos, sin
from sympy.physics.mechanics import *

In [2]:
from __future__ import print_function, division
from sympy import symbols, simplify
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point

In [3]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

In [4]:
from IPython.display import Image

In [111]:
t = Symbol("t")

# Angulos Theta
theta1 = Function("theta1")(t)
theta2 = Function("theta2")(t)
theta3 = Function("theta3")(t)
theta4 = Function("theta4")(t)

# Constante da gravidade
g = dynamicsymbols('g')

# Forcas e Momentos
fx, fy, fz = dynamicsymbols('f_x, f_y, f_z')
taux, tauy, tauz = dynamicsymbols('tau_x, tau_y, tau_z')

# Forcas dos motores
f1,f2= dynamicsymbols('F_1, F_2')

# Angulos Alfa (Angulos do mecanismo tilt)
alpha1 = Function("alpha1")(t)
alpha2 = Function("alpha2")(t)

# Extensão dos bracos do bicopter
b1,b2 = dynamicsymbols('b_1, b_2')

# Extensão dos segmentos do braço robótico
l1, l2, l3, l4 = dynamicsymbols('l1, l2, l3, l4')

#Características inerciais

m1, m2, m3, m4, m5 = dynamicsymbols('m1, m2, m3, m4, m5')
Ix1, Ix2, Ix3, Ix4, Ix5 = dynamicsymbols('Ix1, Ix2, Ix3, Ix4, Ix5')
Iy1, Iy2, Iy3, Iy4, Iy5 = dynamicsymbols('Iy1, Iy2, Iy3, Iy4, Iy5')
Iz1, Iz2, Iz3, Iz4, Iz5 = dynamicsymbols('Iz1, Iz2, Iz3, Iz4, Iz5')

# Ganho de atrito ?
fs1, fs2, fs3, fs4, fs5 = dynamicsymbols('fs1, fs2, fs3, fs4, fs5')


# Variaveis no estado de equilibrio

#estados
theta10, theta20, theta30, theta40 = dynamicsymbols('theta_1_0, theta2_0, theta3_0, theta4_0')
omega10, omega20, omega30, omega40 = dynamicsymbols('omega1_0, omega2_0, omega3_0, omega4_0')
psi10, psi20, psi30, psi40 = dynamicsymbols('psi1_0, psi2_0, psi3_0, psi4_0')

# comandos
alpha10, alpha20= dynamicsymbols('alpha_1_0, alpha2_0')
f10, f20= dynamicsymbols('F_10, F_20')

# simplificacoes

den1, den2, den3, den4 = dynamicsymbols('den1, den2, den3, den4')

In [6]:
# Criando a função de criação da matriz de transformação homogênea

# Matriz Rotação da Transformação
def rt_M(T):
    M = T[0:3,0:3]
    return M

In [7]:
# Matriz Rotação da Transformação
def ch_M(T):
    M=zeros(6)
    M[0:3,0:3] = T[0:3,0:3]
    M[3:6,3:6] = T[0:3,0:3]
    return M

In [8]:
# Matriz da massa e inércia
def mI_M(m,Ix,Iy,Iz):
    M = diag(m, m, m, Ix, Iy, Iz)
    return M

In [9]:
# Ponto de representação homogenea para dimensão 3
def ph_2N(ph):
    p = Matrix(ph[0:3])
    return (p)

In [10]:
# Definindo a matriz jacobiana
def Jc(j1,j2,j3,j4):
    J = zeros(6,1)
    J = J.col_insert(0,j1)
    J = J.col_insert(1,j2)
    J = J.col_insert(2,j3)
    J = J.col_insert(3,j4)
    J.col_del(4)
    return J

In [11]:
# Definindo a nesima coluna da matriz jacobiana
def Ji(m0,op,oi):
    aux1 = m0*Matrix([0,0,1])
    aux2 = aux1.cross(op-oi)
    J = Matrix([aux2, aux1])
    return J

In [12]:
# Definindo a nesima coluna da matriz jacobiana
def JpTi(m0,op,Ti):
    # Calculate the zi= i-1Mi.k
    aux1 = m0*Matrix([0,0,1])
    oi = Matrix([Ti[0:3,3]])
    # Calculate the zi= i-1Mi.k
    aux2 = aux1.cross(op-oi)
    
    J = Matrix([aux2, aux1])
    return J

In [13]:
# Criando a função de criação da matriz de transformação homogênea
# a partir dos parâmetros de Denavit-Hatemberg

#cos = cos
#sin = sin

# Matriz Transformação Homogênea (Homogeous Transformation Matrix)
# Using the Standard Denavit-Hatemberg Convention (not Modified)
def ht_M(theta, d, alpha, a):
    # theta e alpha em radianos
    # d e a em metros
    Rz = Matrix([[cos(theta), -sin(theta), 0, 0],
                   [sin(theta), cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, a],
                   [0, 0, 0, 1]])
    ta = Matrix([[1, 0, 0, d],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = Matrix([[1, 0, 0, 0],
                   [0, cos(alpha), -sin(alpha), 0],
                   [0, sin(alpha), cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

<h2> Declaração dos parâmetros de Denavit-Hatenberg </h2>
<h2> Instanciação das matrizes de transformação homogêneas </h2>

<div>
<img src="img/ModeloModificado.png" width="900">
</div>



|  Elemento | $\alpha_i$ |  $d_i$   | $\theta_i$ |  $r_i$ |
| -------- | -------- | --------| -------- | -------- | 
| $1$ | $\frac{\pi}{2}$   | 0     | $\theta_1$ | $l_1$ |
| $2$ | 0                 | $l_2$ | $\theta_2$ | 0     |
| $3$ | $- \frac{\pi}{2}$ | $l_3$ | $\theta_3$ | 0     |
| $4$ | 0                 |   0   | $\theta_4$ | 0     |

In [14]:
T1 = ht_M(theta1, 0, (pi)/2, l1)
T2 = ht_M(theta2, l2, 0, 0)
T3 = ht_M(theta3, l3, (-pi)/2, 0)
T4 = ht_M(theta4, l4, 0, 0)

In [15]:
#T1 
#T2
#T3
#T4

<h3> Simplificação da matriz T3 a partir da relação entre $\theta_3$ e $\theta_2$</h3>

In [16]:
T3

Matrix([
[cos(theta3),  0, -sin(theta3), l3*cos(theta3)],
[sin(theta3),  0,  cos(theta3), l3*sin(theta3)],
[          0, -1,            0,              0],
[          0,  0,            0,              1]])

In [17]:
T3=T3.subs(theta3,(-pi/2-theta2))

In [18]:
T3

Matrix([
[-sin(theta2),  0,  cos(theta2), -l3*sin(theta2)],
[-cos(theta2),  0, -sin(theta2), -l3*cos(theta2)],
[           0, -1,            0,               0],
[           0,  0,            0,               1]])

<h2> Instanciação das matrizes de transformação homogêneas em realação ao referencial 0 </h2>

In [19]:
T01=T1
T02=T01*T2
T02=trigsimp(T02)
T03=T02*T3
T03=trigsimp(T03)
T04=T03*T4

In [20]:
#(T02)
#(T03)
#(T04)

<h2> Criação das matrizes Jacobianas</h2>

In [21]:
# Definindo a matriz jacobiana
def Jc(j1,j2,j3,j4):
    J = zeros(6,1)
    J = J.col_insert(0,j1)
    J = J.col_insert(1,j2)
    J = J.col_insert(2,j3)
    J = J.col_insert(3,j4)
    J.col_del(4)
    return J

In [22]:
# Criação dos pontos origens
og1=ph_2N(T01*Matrix([0,0,0,1]))
og2=ph_2N(T03*Matrix([-l3/2,0,0,1]))
og3=ph_2N(T03*Matrix([0,0,0,1]))
og4=ph_2N(T04*Matrix([0,0,0,1]))

In [23]:
# Calculo do Jc[1] que é o jacobiano referente ponto centro de gravidade g1
null=Matrix([0,0,0,0,0,0])
J1 = Ji(eye(3),og1, Matrix([0,0,0])) 
Jc1 = Jc(J1,null,null,null)
# Calculo do Jc[2] que é o jacobiano referente ponto centro de gravidade g2
J1 = Ji(eye(3),og2, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og2, Matrix([0,0,0])) 
Jc2 = trigsimp(Jc(J1,J2,null,null))
# Calculo do Jc[3] que é o jacobiano referente ponto centro de gravidade g3
J1 = Ji(eye(3),og3, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og3, Matrix([0,0,0]))
J3 = JpTi(rt_M(T02),og3,T02) 
Jc3 = trigsimp(Jc(J1,J2,J3,null))
# Calculo do Jc[4] que é o jacobiano referente ponto centro de gravidade g4
J1 = Ji(eye(3),og4, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og4, Matrix([0,0,0]))
J3 = JpTi(rt_M(T02),og4,T02) 
J4 = JpTi(rt_M(T03),og4,T03)
Jc4 = trigsimp(Jc(J1,J2,J3,J4))

In [24]:
#Jc1
#Jc2
#Jc3
#Jc4

In [25]:
# Calculo do J
J = Jc(J1,J2,J3,J4)
J

Matrix([
[-l2*sin(theta1)*cos(theta2) - l4*sin(theta4)*cos(theta1),                                                                                      -(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*cos(theta1), -(-l3 - l4*cos(theta4))*cos(theta1),                                   -l4*sin(theta1)*cos(theta4)],
[ l2*cos(theta1)*cos(theta2) - l4*sin(theta1)*sin(theta4),                                                                                      -(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*sin(theta1), -(-l3 - l4*cos(theta4))*sin(theta1),                                    l4*cos(theta1)*cos(theta4)],
[                                                       0, (l2*sin(theta1)*cos(theta2) + l4*sin(theta4)*cos(theta1))*sin(theta1) + (l2*cos(theta1)*cos(theta2) - l4*sin(theta1)*sin(theta4))*cos(theta1),                                   0, l4*sin(theta1)**2*sin(theta4) + l4*sin(theta4)*cos(theta1)**2],
[                                                       0,                        

<h2> Criação das matrizes de Inércia</h2>

In [26]:
mI_1 = mI_M(m1,Ix1,Iy1,Iz1)
mI_2 = mI_M(m2,Ix2,Iy2,Iz2)
mI_3 = mI_M(m3,Ix3,Iy3,Iz3)
mI_4 = mI_M(m4,Ix4,Iy4,Iz4)
mI_4

Matrix([
[m4,  0,  0,   0,   0,   0],
[ 0, m4,  0,   0,   0,   0],
[ 0,  0, m4,   0,   0,   0],
[ 0,  0,  0, Ix4,   0,   0],
[ 0,  0,  0,   0, Iy4,   0],
[ 0,  0,  0,   0,   0, Iz4]])

In [27]:
# Build the Di matriz
def Di_M(Jc_gi,mI):
    M = (Jc_gi.T)*mI*Jc_gi;
    return M

In [28]:

D1 = Di_M(Jc1,mI_1)
D2 = trigsimp(Di_M(Jc2,mI_2))
D3 = trigsimp(Di_M(Jc3,mI_3))
D4 = trigsimp(Di_M(Jc4,mI_4))
Dtotal = D1+D2+D3+D4

In [29]:
#Dtotal

<h2> Calculo da Energia Cinética </h2>

In [30]:
q=Matrix([theta1,theta2,theta3,theta4])
q_dot= diff(q,t)

In [31]:
K = 1/2* q_dot.T * Dtotal * q_dot

<h2> Calculo da Energia Potencial</h2>

In [32]:
og1=T01*Matrix([0,0,0,1])
og2=T03*Matrix([-l3/2,0,0,1])
og3=T03*Matrix([0,0,0,1])
og4=T04*Matrix([0,0,0,1])

In [33]:
P1 = m1* g * Matrix([0,0,1,0]).T * og1
P2 = m2* g * Matrix([0,0,1,0]).T * og2
P3 = m3* g * Matrix([0,0,1,0]).T * og3
P4 = m4* g * Matrix([0,0,1,0]).T * og4

In [34]:
Ptotal = P1+ P2 + P3 + P4
#Ptotal

<h2> Calculo do Lagrangiano </h2>

In [35]:
L= K-Ptotal

<h1> Definição das variáveis articulares </h1>
\begin{align}
    q &= \begin{bmatrix}
    \theta_1 \\
    \theta_2 \\
    \theta_3 \\
    \theta_4
    \end{bmatrix}\label{eq:q}\\ 
    \dot{q} &= \begin{bmatrix}
    \dot{\theta_1} \\
    \dot{\theta_2} \\
    \dot{\theta_3} \\
    \dot{\theta_4}\end{bmatrix}\label{eq:qdot}\\
    \ddot{q} &= \begin{bmatrix}
    \ddot{\theta_1} \\
    \ddot{\theta_2} \\
    \ddot{\theta_3} \\
    \ddot{\theta_4}\end{bmatrix}\label{eq:qddot}
\end{align}

In [36]:
q=Matrix([theta1,theta2,theta3,theta4])
q_dot= diff(q,t)
q_ddot= diff(q_dot,t)

<h2> Calculo dos termos da equaçoes de Euler Lagrange </h2>

A partir da formulação das equações de Euler-Lagrange o modelo dinâmico de braço robóticos é definido por 

\begin{equation}
     D(q)\ddot{q}+C(q,\ddot{q})+G(q)=\Gamma
     \label{eq:EulerLagrange}
\end{equation} 

em que o termo $D(q)\ddot{q}$ designa as contribuições inerciais, $C(q,\dot{q})$ corresponde as contribuições centrífugas e de Coriolis, $G(q)$ representa as contribuições da gravidade, $\Gamma$ diz respeito aos esforços articulares e $q$, $\dot{q}$ e $\ddot{q}$ são as posições, velocidades  e as acelerações das juntas que compõem o conjunto, respectivamente.

Matematicamente o termo $D(q)$ é definido por

\begin{equation}
    D(q)= \sum_{i=1}^4[J_i(G_i,q)]^T\cdot\begin{bmatrix}
    m_i & 0 & 0 & \emptyset_{1x3}\\
    0 & m_i & 0 & \emptyset_{1x3}\\
    0 & 0 & m_i & \emptyset_{1x3}\\
    \emptyset_{3x3} & \emptyset_{3x3} & \emptyset_{3x3} & \bar{I}_i
    \end{bmatrix}\cdot J_i(G_i,q)
     \label{eq:D_euler_lagrange}
\end{equation}

sendo que $G_i$ o centro de gravidade do elemento $i$, $\emptyset_{3x3}$ uma matriz de zeros, $\emptyset_{1x3}$ um vetor linha de zeros, $\bar{I}_i$ a matriz de inércia em $G_i$ e $J_i(G_i,q)$ a i-ésima coluna da matriz jacobiana cinemática do centro de gravidade do i-ésimo elemento. 

Como todas as juntas que compõem o conjunto braço robótico-bicóptero são de revolução, então $J_i(G_i,q)$ \cite{Park2017} vale 

\begin{equation}
    J_i(G_i)=
    \begin{bmatrix}
    z_{i-1}\times \overrightarrow{O_{i-1} O_{G_i}}\\
    z_{i-1}
    \end{bmatrix}
    \label{eq:jicgi}
\end{equation}

O termo $C(q,\dot{q})$ é definido por

\begin{equation}
     C(q, \dot{q})=\left( \sum_{k=1}^4 \frac{\partial D(q) }{\partial q_k}\dot{q}_k \right) \dot{q} - \begin{bmatrix}
    \frac{1}{2}\dot{q}^T\frac{\partial D(q)}{\partial q_1\dot{q}}\\
    \vdots\\
    \frac{1}{2}\dot{q}^T\frac{\partial D(q)}{\partial q_4\dot{q}}
    \end{bmatrix}
    \label{eq:C_euler_lagrange}
\end{equation}

E o termo $G(q)$ vale

\begin{equation}
    G(q)= \sum_{i=1}^4 m_i g  \begin{bmatrix}
    \frac{\partial zG_{i/R0}}{\partial q_1}\\
    \vdots\\
     \frac{\partial zG_{i/R0}}{\partial q_4}
    \end{bmatrix}
    \label{eq:Q_euler_lagrange}
\end{equation}


In [37]:
# Termo D(q) contribuicao inercial vale D_total ja calculado precedemente
#Dtotal
#q[1]
#diff(Dtotal,q[1])*q_dot[1]

In [38]:
def C_t(q,q_dot,D_total):
    C_1 = zeros(4,4)
    C_2 = zeros(4,1)

    for n in range(4):
        # Part 1 = dD(q)/dt = sum (partial D(q)/partial qi)q_dot
        C_1= C_1 + (diff(D_total,q[n])*q_dot[n])

    C_1 = C_1*q_dot

    for n in range(4):
        # Part 2 = column matrix (1/2)*(q_dot)^T.(partial D(q)/partial qi).q_do
        C_2[n] = (1/2)*((q_dot.T)*diff(D_total,q[n])*q_dot)

    C = C_1 - C_2;
    return C

In [39]:
# Termo C(q,qdot) contribuicao inercial
Ctotal = C_t(q, q_dot, Dtotal)
#Ctotal

In [40]:
def G_t(q,P_total):
    G = zeros(4,1)
    for n in range(4):
        G[n] = diff(P_total,q[n])
    return G

In [41]:
# Termo G(q)  vale as contribuicoes da gravidade 
Gtotal = G_t(q,Ptotal)
Gtotal

Matrix([
[                                                              0],
[g*l2*m2*cos(theta2) + g*l2*m3*cos(theta2) + g*l2*m4*cos(theta2)],
[                                                              0],
[                                            g*l4*m4*sin(theta4)]])

In [42]:
def v_i(M0,Tp,Ti):
    # Calculate the zi= i-1Mi.k
    aux1 = M0*Matrix([0,0,1])

    Op = Tp[0:3,3]
    Oi = Ti[0:3,3]

    # Cross Product  zi x (Op-Oi)
    aux2 = (aux1.cross(Op-Oi))
    
    # Building cinematic tensor for pivot joints
    
    v = Matrix([aux2, aux1])
    return v 

In [43]:
#Matriz rotacional 0^M_0
# Calculate the zi= i-1Mi.k 
J1 = v_i(eye(3), T04, zeros(4,4))

J2 = trigsimp(v_i(rt_M(T01), T04, T01))

J3 = v_i(rt_M(T02),T04,T02)

J4 = v_i(rt_M(T03),T04,T03)

JcP4 = Jc(J1,J2,J3,J4)

v = (JcP4*Matrix([0,1,-1,0])).subs({theta1: 0, theta2:0,theta4: 0})
#(JcP4.T)
#v
#(JcP4.T)*Matrix([-fx,0,0,0,0,0])
#Matriz jacobiano cinemática completa
#JcP4

<h1> Calculo de $\Gamma_e$ </h1>

\begin{equation}
    \Gamma_e= \begin{bmatrix}{}^{4}J_n(P,\textbf{q}) \end{bmatrix}^T \cdot {}^4\mathcal{T}_e(P)
    \label{eq:Gamma_eP}
\end{equation}

sendo ${}^{4}J_n(P,q)$ a n-ésima coluna da matriz jacobiana cinemática definida no referencial $R_4$ do elemento $S_n$ ao ponto de aplicação $P$ da força exterior, que no nosso caso corresponde ao ponto origem do referencial do bicóptero, e ${}^4\mathcal{T}_e(P)$ o vetor dos esforços aplicadas pelo órgão terminal do robô sobre o ambiente expresso no referencial $R_4$.

\begin{equation}
    {}^4\mathcal{T}_e(P) =\begin{bmatrix}
    f_x & f_y & f_z & \tau_x & \tau_y &\tau_z
    \end{bmatrix}^T
    \label{eq:t_e}
\end{equation}

em que $f_x,\ f_y,\ f_z$ correspondem as componentes da força total e $\tau_x,\ \tau_y,\ \tau_z$ as componentes do momento total nos eixos $X_{j_4},\ Y_{j_4}$ e $Z_{j_4}$ respectivamente.


<h2> Calculo do esforcos de interaçao $\Gamma_e$ </h2>
<ol>
  <li>Representar os tensores cinematicos em relacao a P</li>
  <li>Identificar o vetores OiP sendo P o ponto do referencial bicoptero</li>
  <li>Multiplicar por i-1Mi (matriz de rotacao) </li>
  <li>fazer o produto vetorial entre Zi x (i-1Mi * OiP)</li>
  <li>Montar o tensor</li>
</ol>

In [44]:
#Forca expressa no referencial 4
#tau_e:=<<f[x],f[y],f[z],tau[x],tau[y],tau[z]>>:
tau_e = Matrix([fx,fy,0,0,0,tauz])
Gamma_e = JcP4.T*(tau_e)
Gamma_e.subs({theta2:0,theta1:0,theta4:0})


Matrix([
[ f_y*l2 + tau_z],
[-(-l3 - l4)*f_x],
[-(-l3 - l4)*f_x],
[         f_y*l4]])

In [45]:
# Para entao representar o vetor esforco em outro referencial é necessario as seguintes formulacoes

# Para entender indica-se o exemplo da maçã do link https://modernrobotics.northwestern.edu/nu-gm-book-resource/3-4-wrenches/#department
#Ttest:=eval(ht_M(0,-(ele),0,0));
#tau_e:=<<0,0,0,0,-mg,0>>;
#p:=v_2so3(<<Ttest(1,4),Ttest(2,4),Ttest(3,4)>>);
#T_adj(Ttest,rt_M(Ttest));
#tau_f:=LinearAlgebra[Transpose](T_adj(Ttest)).(tau_e);


#representacao de um vetor em so3
def v_2so3(v):
    p = zeros(3)
    p[0,1] = -v[2,0]
    p[0,2] = v[1,0]
    p[1,2] = -v[0,0]
    p[1,0] = v[2,0]
    p[2,0] = -v[1,0]
    p[2,1] = v[0,0]
    return p

#representacao adjunta da matriz Transformacao homogenea
def T_adj(Thom):
    p = v_2so3(Thom[0:3,3])

    K = p*rt_M(Thom)
    M = zeros(6)

    M[0:3,0:3] = Thom[0:3,0:3]
    M[3:6,0:6] = Matrix([K[0:3,0:3]]).col_insert(3,Thom[0:3,0:3]) 

    return M

#Inversa da matriz homogênea
def inv_Thom(Thom):
    Ka = (rt_M(Thom)).T
    Ke = -Ka*Thom[0:3,3] 

    M = Ka[0:3,0:3]
    M = M.col_insert(3,Ke[0:3,0])

    M = M.row_insert(4, Matrix([0,0,0,1]).T)
    return M

In [46]:
#Calcula-se a matrix inversa (T_0_4) da matriz transformacao homogenea T_04 
T40 =inv_Thom(T04)

#eval(T_4_[0],{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});

#Passa-se o vetor 3x1 p da entao matriz inversa para s03
p=v_2so3(T40[0:3,3])

#Finalmente calcula-se a matrix adjunta de T_4_0
MAdjT40 = T_adj(T40)
#T40
#p
#MAdjT40
#eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});

#LinearAlgebra[Transpose](eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0})).<<0,0,tau[z],f[x],f[y],0>> := LinearAlgebra[Transpose](eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0})).<<0,0,tau[z],f[x],f[y],0>> ;

#Constroi-se uma matrix capaz de colocar os elementos momentos nas ultimas 3 posicoes do vetor 6x1 e colocar as forcas nas 3 primeiras posicoes do vetor
MF_M = zeros(6)
MF_M[0,3] =1
MF_M[1,4] =1
MF_M[2,5] =1
MF_M[3,0] =1
MF_M[4,1] =1
MF_M[5,2] =1


<h2>Mudança de base da matriz jacobiana para o referencial R4<\h2>
    
\begin{equation}
     {}^{4}J_n(P,q)= \begin{bmatrix}
     {}^{4}M_0 & \emptyset_{3x3}  \\
     \emptyset_{3x3} & {}^{4}M_0\end{bmatrix} \cdot {}^{0}J_n(P,q)
    \label{eq:mudança de base_Jcp}
\end{equation}

em que ${}^{4}M_0$ corresponde à matriz de rotação que compõe a matriz de transformação homogênea ${}^{4}T_0= ({}^{0}T_4)^{-1}$.

In [47]:
#Perceba que inicialmente o vetor tau_e é da maneira <<momentos, forcas>>
# Alem disso para o nosso aplicacao taux=tauy=fz=0
tau_e = Matrix([0,0,tauz,fx,fy,0])

#Aqui mostra-se como os esforcos em P sao traduzidos no referencial 0
#eval(MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6),{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0}):
PJ4 = trigsimp(ch_M(rt_M(T40))*JcP4);

#aux:=MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6):

#LinearAlgebra[Transpose](J_cP[4]);

#Calcula-se Gamma_e
#Gamma_e:=LinearAlgebra[Transpose](J_cP[4]).MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6):

Gamma_e = PJ4.T*MF_M*(tau_e)

#Nosso caso queremos Gamma_b
Gamma_b = Gamma_e

Gamma_b = trigsimp(Gamma_b)
Gamma_b
#Visualização dos esforços em cada junta na posicao neutra 
#MF_M.<<0,0,tau[z],f[x],f[y],0>>;
#eval(MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6),{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});
#eval(Gamma_b,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});
PJ4.T

Matrix([
[ l2*sin(theta4)*cos(theta2), l2*cos(theta2)*cos(theta4),                       -l4*sin(theta4), -cos(theta4),  sin(theta4), 0],
[-l2*cos(theta2)*cos(theta4), l2*sin(theta4)*cos(theta2), -l2*sin(theta2) + l3 + l4*cos(theta4), -sin(theta4), -cos(theta4), 0],
[                          0,                          0,                   l3 + l4*cos(theta4), -sin(theta4), -cos(theta4), 0],
[                          0,                         l4,                                     0,            0,            0, 1]])

<h1> Traduzindo as variaveis de entrada com o mecanismo de vetorização</h1>

<div>
<img src="img/forcas_bicoptero.png" width="500">
</div>

\begin{equation}
   \begin{bmatrix}
    f_x\\
    f_y\\
    0  \\
    0  \\
    0 \\
    \tau_z \end{bmatrix} =
    \begin{bmatrix}
    -c(\alpha_1) & -c(\alpha_2)\\
    s(\alpha_1) &
    -s(\alpha_2)\\
    0 & 0 \\
    0 & 0 \\
    0 & 0 \\
    b_1c(\alpha_1) & -b_2c(\alpha_2) 
    \end{bmatrix}
    \cdot
    \begin{bmatrix}
    F_{e_1}\\
    F_{e_2}
    \end{bmatrix}
    \label{eq:vetorizacao}
\end{equation}

In [48]:
tilt = Matrix([[-cos(alpha1), -cos(alpha2)],[sin(alpha1), -sin(alpha2)], zeros(3,2), [b1*cos(alpha1), -b2*cos(alpha2)]])
forces= Matrix([f1,f2])
tilt

Matrix([
[   -cos(alpha1),     -cos(alpha2)],
[    sin(alpha1),     -sin(alpha2)],
[              0,                0],
[              0,                0],
[              0,                0],
[b_1*cos(alpha1), -b_2*cos(alpha2)]])

In [49]:
#Calcula-se Gamma_e
Gamma_e = PJ4.T*tilt*forces

#Nosso caso queremos Gamma_b
Gamma_b = Gamma_e

Gamma_b = trigsimp(Gamma_b)
Gamma_b
Gamma_b.subs({theta1:0,theta2:0,theta4:0,alpha1:0,alpha2:0})

Matrix([
[                0],
[   (F_1 + F_2)*l2],
[                0],
[F_1*b_1 - F_2*b_2]])

<h1> Esforços realizados pelo contrapeso $\Gamma_c$ </h1>

\begin{equation}
     \Gamma_c=\sum^n_{i=1}\Gamma_m,i= \begin{bmatrix} 
     m_1g\frac{\partial zG_{1/R0}}{\partial q_1} & + & m_2g\frac{\partial zG_{2/R0}}{\partial q_1} & + & ... & + & m_ng\frac{\partial zG_{n/R0}}{\partial q_1}\\
    & & m_2g\frac{\partial zG_{2/R0}}{\partial q_2} & + & ... & + & m_ng\frac{\partial zG_{n/R0}}{\partial q_2}\\
    & & & & & \ddots & \vdots\\
    & & & & & & m_ng\frac{\partial zG_{n/R0}}{\partial q_n}
     \end{bmatrix}
    \label{eq:tensorpesanteur_formula}
\end{equation}


In [50]:
Gamma_c=zeros(4,1)
# Nao realizamos para o peso do elemento final que é o drone 
Gamma_c[0] = diff(P1,q[0])+diff(P2,q[0])+diff(P3,q[0])#+diff(P4,q[0])
Gamma_c[1] = diff(P2,q[1])+diff(P3,q[1])#+diff(P4,q[1])
Gamma_c[2] = diff(P3,q[2])#+diff(P4,q[2])
    
Gamma_c

Matrix([
[                                        0],
[g*l2*m2*cos(theta2) + g*l2*m3*cos(theta2)],
[                                        0],
[                                        0]])

<h1> Esforços devido ao atrito viscoso $\Gamma_f$ </h1>

Os coeficientes de atrito viscoso em torno dos ângulos de rotação $\theta_2$ e $\theta_4$ foram determinados seguindo o procedimento descrito no Apêndice.  Já o atrito em torno de $\theta_1$ foi desconsiderado, pois não foi viável determiná-lo experimentalmente de forma segura. Logo $\Gamma_f( \dot{q})$ é descrito por

\begin{equation}
    \Gamma_f( \dot{q})= \begin{bmatrix}
    0\\
    f_{s_2}\dot{\theta}_2\\
    0\\
    f_{s_4}\dot{\theta}_4
    \end{bmatrix}
    \label{eq:gamma_f}
\end{equation}



In [51]:
Gamma_f = diag(fs1, fs2, fs3, fs4)*q_dot
Gamma_f = Gamma_f.subs({fs1:0,fs3:0})
Gamma_f

Matrix([
[          0],
[fs2*theta2'],
[          0],
[fs4*theta4']])

<h2> Equação Total do Movimento </h2>

\begin{equation}
    \emptyset_{4x6} =D(q)\ddot{q} + C(q,\dot{q}) + G(q) + \Gamma_f(\dot{q}) - \Gamma_c - \Gamma_b 
    \label{eq:modele_dyn_fb}
\end{equation}


In [52]:
Mdf=Dtotal*q_ddot+Ctotal+Gtotal+Gamma_f-Gamma_b-Gamma_c

<h2>Iniciando Equacao Solução</h2>

A equação relativa a $\ddot{\theta_3}$ é completamente ignorada, Eq tem dimensão igual a $3\times1$ devido a relação entre $\theta_3$ e $\theta_2$

In [337]:
Eq=zeros(3,1)

In [338]:
#Isolando as variaveis q_ddot
Eq[0]=solve(Mdf[0], q_ddot[0])  #Relativo a Theta dois pontos 1
Eq[1]=solve(Mdf[1], q_ddot[1])  #Relativo a Theta dois pontos 2
#Eq[2]=solve(Mdf[2], q_ddot[2])  #Relativo a Theta dois pontos 3
Eq[2]=solve(Mdf[3], q_ddot[3])  #Relativo a Theta dois pontos 4

<h2>Realizando a Linearização</h2>

<h2> Linearização (Método de Newton) </h2>

![title](img/linearizacao.png)

<a href="http://sites.poli.usp.br/d/PMR2400/3-%20Lineariza%C3%A7%C3%A3o%20de%20sistemas.pdf"> Material Referência </a>


<h2>Iniciando o vetor Estado e Comando</h2>
Como o vetor de estados vale

$\begin{equation} \vec{x}=\begin{bmatrix} 
\theta_{1}\\
\theta_{2}\\
\theta_{4}\\
\dot{\theta}_{1}\\
\dot{\theta}_{2}\\
\dot{\theta}_{4}\end{bmatrix}
\end{equation}$

O vetor dos comandos vale

$\begin{equation}\vec{u}= \begin{bmatrix} 
\alpha_1\\
\alpha_2\\
F_1\\
F_2\end{bmatrix}
\end{equation}$



In [354]:
#states=Matrix([theta1, theta2, theta4, dot theta1 dottheta2 dottheta4])
states=q.row_insert(4, q_dot)
states.row_del(6)
states.row_del(2)
states

Matrix([
[ theta1],
[ theta2],
[ theta4],
[theta1'],
[theta2'],
[theta4']])

In [415]:
inputs=Matrix([f1,f2,alpha1,alpha2])
inputs

Matrix([
[   F_1],
[   F_2],
[alpha1],
[alpha2]])

In [416]:
len(inputs)#4
len(states)#6
len(Eq)#3

3

<h2>Iniciando o vetor estado inicial</h2>

In [417]:
trim_inputs={f1:f10,f2:f20,alpha1:alpha10,alpha2:alpha20}

trim_position={theta1:theta10,theta2:theta20,theta3:theta30,theta4:theta40}
trim_speed={diff(theta10,t):omega10,diff(theta20,t):omega20,diff(theta30,t):omega30,diff(theta40,t):omega40}
trim_acel={diff(diff(theta10,t),t):psi10,diff(diff(theta20,t),t):psi20,diff(diff(theta30,t),t):psi30,diff(diff(theta40,t),t):psi40}

#Estados iniciais
val_pos={theta10: 0.00,theta20: 0.00,theta30: -(pi/2),theta40: 0.00}
val_speed={omega10: 0.00,omega20: 0.00,omega30: 0.00,omega40: 0.00}
val_acel={psi10: 0.00,psi20: 0.00,psi30: 0.00,psi40: 0.00}
val_inputs={alpha10: 0.00,alpha20: 0.00} #f10: g*m4/2,f20: g*m4/2}

In [418]:
Eq_trim=zeros(len(Eq),1)

In [419]:
for i in range (len(Eq)):
#    Eq_trim[i]=Eq[i].subs(Trim).subs(states_value).subs(inputs_value)
    Eq_trim[i]=Eq[i].subs(trim_inputs).subs(val_inputs).subs(trim_position).subs(trim_acel).subs(trim_speed).subs(val_pos).subs(val_acel).subs(val_speed).factor().nsimplify()

In [420]:
Eq_trim
#Observe que se b1=b2 e f10=f20=(1/2)g*m4 tudo é todas as acelerações são iguais a zero o que significa que está em equilibrio

Matrix([
[                                                                                                                                                                                                                                             0],
[(F_10 + F_20 - g*m4)*l2/(2*(Iy2/2 + Iy3/2 + Iy4/2 + l1**2*m2/2 + l1**2*m3/2 + l1**2*m4/2 - l1*l3*m2/2 - l1*l3*m3 - l1*l3*m4 - l1*l4*m4 + l2**2*m2/2 + l2**2*m3/2 + l2**2*m4/2 + l3**2*m2/8 + l3**2*m3/2 + l3**2*m4/2 + l3*l4*m4 + l4**2*m4/2))],
[                                                                                                                                                                                                        (F_10*b_1 - F_20*b_2)/(Ix4 + l4**2*m4)]])

In [421]:
#Eq

<h1>Dimensionamento correto das matrizes A e B</h1>

a matrix $A$ deve ser de dimensão $6\times6$ e $B$ de dimensão $4\times6$

sendo que as primeiras 3 linhas da matriz A são

$\begin{equation}
\begin{bmatrix}
0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0\\
0& 0 & 0 & 0 & 0 & 1\\
\end{bmatrix}\end{equation}$

e as primeiras 3 linhas da matriz B são

$\begin{equation}
\begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0& 0 & 0 & 0 \\
\end{bmatrix}\end{equation}$


In [422]:
A=zeros(3,3).col_insert(3, eye(3))
A=A.row_insert(4, zeros(3,6))
A

Matrix([
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0]])

In [423]:
B = zeros(6, 4) 
inputs_value={alpha10: 0,alpha20: 0} #f10: g*m4/2,f20: g*m4/2}
B

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [424]:
for i in range(len(Eq)):
    for j in range(len(states)):
         A[i+3,j]=( diff( Eq[i], states[j] ) ).subs(trim_inputs).subs(val_inputs).subs(trim_position).subs(trim_acel).subs(trim_speed).subs(val_pos).subs(val_acel).subs(val_speed).factor().nsimplify()


In [425]:
for i in range(len(Eq)):
    for j in range(len(inputs)):
        B[i+3,j]= ( diff( Eq[i], inputs[j] ) ).subs(trim_inputs).subs(val_inputs).subs(trim_position).subs(trim_acel).subs(trim_speed).subs(val_pos).subs(val_acel).subs(val_speed).factor().nsimplify()

In [426]:
A

Matrix([
[0,                                                                                                                                                                                                                                                                                                               0,                                                                          0, 1,                                                                                                                                                                                                                           0,                     0],
[0,                                                                                                                                                                                                                                                                                                               0,                                                  

In [427]:
B

Matrix([
[                                                                                                                                                                                                                        0,                                                                                                                                                                                                                         0,                                                                0,                                                                 0],
[                                                                                                                                                                                                                        0,                                                                                                                                                                                                       

In [264]:
#delta_states=Matrix([(theta1-theta10), (theta2-theta20), (theta3-theta30), (theta4-theta40)])
#delta_inputs=Matrix([(alpha1-alpha10), (alpha2-alpha20), (f1-f10), (f2-f20)])
#Sol=A*delta_states+B*delta_inputs
#Sol

<h2>Particularizando</h2>

Seja

$\begin{align}
l4&=0\\
F_{1_0}&=F_{2_0}=\frac{1}{2}m_4g
\end{align}$


In [475]:
simpl1=({l4:0})#,f10: g*m4/2,f20: g*m4/2}) # forcas de trimagem
Asimp=A.subs(simpl1)
Bsimp=B.subs(simpl1)

In [476]:
Asimp

Matrix([
[0,                                                                                                                                                                                                                                                                    0,                                                                          0, 1,                                                                                                                                                                                        0,        0],
[0,                                                                                                                                                                                                                                                                    0,                                                                          0, 0,                                                                                                         

In [477]:
Bsimp

Matrix([
[                                                                                                                                                                                     0,                                                                                                                                                                                      0,                                                                0,                                                                 0],
[                                                                                                                                                                                     0,                                                                                                                                                                                      0,                                                                0,                                                      

In [453]:
n,denominator2 = fraction(Asimp[4,4])
Asimp[4,4]=n/denominator2
Asimp[4,4]

-fs2/(Iy2 + Iy3 + Iy4 + l1**2*m2 + l1**2*m3 + l1**2*m4 - l1*l3*m2 - 2*l1*l3*m3 - 2*l1*l3*m4 + l2**2*m2 + l2**2*m3 + l2**2*m4 + l3**2*m2/4 + l3**2*m3 + l3**2*m4)

In [454]:
n,denominator2 = fraction(Bsimp[4,0])
Bsimp[4,0]=n/denominator2
n,denominator2 = fraction(Bsimp[4,1])
Bsimp[4,1]=n/denominator2

E simplificando a notação

$\begin{align}
    den_1 & = Iz_{1} + Iz_{2} + Iz_{3} + Iz_{4} + l^{2}_{2} m_{2} + l^{2}_{2} m_{3} + l^{2}_{2} m_{4} \\
    den_2 & = Iy_{2} + Iy_{3} + Iy_{4} + l^{2}_{1} m_{2} + l^{2}_{1} m_{3} + l^{2}_{1} m_{4} - l_{1} l_{3} m_{2} - 2 l_{1} l_{3} m_{3} - 2 l_{1} l_{3} m_{4} + l^{2}_{2} m_{2} + l^{2}_{2} m_{3} + l^{2}_{2} m_{4} + \frac{l^{2}_{3} m_{2}}{4} + l^{2}_{3} m_{3} + l^{2}_{3} m_{4} \\
    den_3 & = 2 \left(\frac{Iy_{2}}{2} + \frac{Iy_{3}}{2} + \frac{Iy_{4}}{2} + \frac{l^{2}_{1} m_{2}}{2} + \frac{l^{2}_{1} m_{3}}{2} + \frac{l^{2}_{1} m_{4}}{2} - \frac{l_{1} l_{3} m_{2}}{2} - l_{1} l_{3} m_{3} - l_{1} l_{3} m_{4} + \frac{l^{2}_{2} m_{2}}{2} + \frac{l^{2}_{2} m_{3}}{2} + \frac{l^{2}_{2} m_{4}}{2} + \frac{l^{2}_{3} m_{2}}{8} + \frac{l^{2}_{3} m_{3}}{2} + \frac{l^{2}_{3} m_{4}}{2}\right)^{2} \\& = 2(\frac{1}{2}den_2)^2
\end{align}$

In [455]:
#recuperando os denominadores
n,denominator1 = fraction(Asimp[3,2])  
n,denominator2 = fraction(Asimp[4,4])
n,denominator3 = fraction(Asimp[4,1])

In [474]:
#denominator1
denominator2

Iy2 + Iy3 + Iy4 + l1**2*m2 + l1**2*m3 + l1**2*m4 - l1*l3*m2 - 2*l1*l3*m3 - 2*l1*l3*m4 + l2**2*m2 + l2**2*m3 + l2**2*m4 + l3**2*m2/4 + l3**2*m3 + l3**2*m4

In [470]:
#(2*pow(((0.5*denominator2).nsimplify()),2))=den3
denominator3

2*(Iy2/2 + Iy3/2 + Iy4/2 + l1**2*m2/2 + l1**2*m3/2 + l1**2*m4/2 - l1*l3*m2/2 - l1*l3*m3 - l1*l3*m4 + l2**2*m2/2 + l2**2*m3/2 + l2**2*m4/2 + l3**2*m2/8 + l3**2*m3/2 + l3**2*m4/2)**2

In [478]:
simpl2=({denominator1:den1,denominator2:den2,denominator3:den3})#,denominator3:den3})
Asimp=Asimp.subs(simpl2)
Bsimp=Bsimp.subs(simpl2)

In [479]:
Asimp

Matrix([
[0,                                                                                  0,                      0, 1,         0,        0],
[0,                                                                                  0,                      0, 0,         1,        0],
[0,                                                                                  0,                      0, 0,         0,        1],
[0,                                                                                  0, -(F_10 + F_20)*l2/den1, 0,         0,        0],
[0, -(F_10 + F_20 - g*m4)*(l1*m2 + l1*m3 + l1*m4 - l3*m2/2 - l3*m3 - l3*m4)*l2**2/den3,                      0, 0, -fs2/den2,        0],
[0,                                                                                  0,                      0, 0,         0, -fs4/Ix4]])

In [480]:
Bsimp

Matrix([
[      0,        0,            0,             0],
[      0,        0,            0,             0],
[      0,        0,            0,             0],
[      0,        0, F_10*l2/den1, -F_20*l2/den1],
[l2/den2,  l2/den2,            0,             0],
[b_1/Ix4, -b_2/Ix4,            0,             0]])